In [1]:
import os 
import getpass
from openai import OpenAI

In [2]:
if not os.environ.get("OPENROUTER_API_KEY"):
    os.environ["OPENROUTER_API_KEY"] = getpass.getpass("Enter Your OpenRouter API Key here:")

In [11]:
%pip install langchain


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\autoManim\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [16]:
%pip install "langchain[openai]"

  Using cached tiktoken-0.9.0-cp310-cp310-win_amd64.whl (894 kB)
  Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl (274 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\autoManim\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [21]:
import langchain
from langchain_openai.chat_models.base import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain

In [37]:
system_template = """
You are “AutoManim Assistant,” an AI expert whose *only* job is to generate complete, runnable Python scripts for 2D animations using the Manim Community v0.19 library. Never output HTML, CSS, or any other language.

Rules:
1. **Always** wrap your entire response in a single ```python``` code block.
2. The script must include:
   - `from manim import *`  
   - A `class MyScene(Scene):` definition with a `construct(self)` method implementing the animation steps.  
   - A “main guard” at the bottom to instantiate and render the scene:

    ```python
    if __name__ == "__main__":
        scene = MyScene()
        scene.render()
    ```
3. Name the scene class descriptively (e.g. `ExpandCircle`, `RotateSquare`, etc.).
4. Use clear, idiomatic Manim patterns (e.g. `self.play(...)`, `.animate`, run_time parameters).
5. Do **not** include any prose or comments outside of the code block—unless the user explicitly requests an explanation.
6. If the user’s request is ambiguous, ask a clarifying question instead of guessing.

Example:

User: “Make a circle expand from radius 0 to 2 over 2 seconds and save the output.”  
Assistant:
```python
from manim import *

class ExpandCircle(Scene):
    def construct(self):
        circle = Circle(radius=0)
        self.play(circle.animate.set(radius=2), run_time=2)

if __name__ == "__main__":
    scene = ExpandCircle()
    scene.render()

"""

systemMessage = SystemMessagePromptTemplate.from_template(system_template)

human_template = "Question : {question}"
human_message = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([systemMessage,human_message])

llm = ChatOpenAI(
    openai_api_key = os.environ.get("OPENROUTER_API_KEY"),
    openai_api_base = "https://openrouter.ai/api/v1",
    model_name = "deepseek/deepseek-prover-v2:free"
)

llm_chain = LLMChain(prompt = chat_prompt , llm = llm)
question = """
Create a complete, runnable Manim Community v0.19 Python script named `BouncingBallsInCircle` that does the following:

1. Draws a large static circle boundary (radius 3) centered on screen.
2. Spawns N=10 small solid balls (radius 0.2) at random positions _inside_ the big circle, each with a random initial velocity vector.
3. Simulates elastic collisions:
   - Between each ball and the circular wall: reflect the velocity vector when a ball hits the boundary.
   - Between balls: exchange velocities on overlap (perfectly elastic).
4. Uses Manim’s updaters or a custom `PhysicsBall` class so that:
   - Positions are updated frame‑by‑frame according to velocity and `dt`.
   - Velocities are adjusted on collisions.
5. Runs the simulation for `run_time=10` seconds at 60 FPS.
6. Includes all imports, a descriptive `class BouncingBallsInCircle(Scene):` with a `construct()` method, and a `if __name__ == "__main__": ...` guard to render and save the video.
7. Wraps the entire response in one ```python``` code block—no extra prose.

Make sure the balls never escape the circle and that collisions conserve speed.  
"""
print(llm_chain.run({"question" : question}))

```python
from manim import *
import numpy as np
from numpy.random import uniform, randint

class PhysicsBall:
    def __init__(self, scene, radius=0.2, color=BLUE, boundary_radius=3, speed=1):
        self.scene = scene
        self.radius = radius
        self.color = color
        self.boundary_radius = boundary_radius
        self.speed = speed
        self.position = self.random_position()
        self.velocity = self.random_velocity()
        self.mobject = Circle(radius=radius, color=color).move_to(self.position)
        self.scene.add(self.mobject)
        self.mobject.add_updater(self.update)

    def random_position(self):
        angle = uniform(0, 2 * np.pi)
        radius = uniform(0, self.boundary_radius - self.radius)
        return np.array([radius * np.cos(angle), radius * np.sin(angle), 0])

    def random_velocity(self):
        angle = uniform(0, 2 * np.pi)
        return self.speed * np.array([np.cos(angle), np.sin(angle), 0])

    def update(self, mobject, dt):
  